In [1]:
# process data and save to memory as variables, not storage
import sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from PyPDF2 import PdfMerger
from tqdm import trange

2023-08-11 04:39:35.100986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 04:39:35.763298: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
wireplane='U'

In [3]:
# takes full raw data and extracts waveform of length nticks
def extract_wave(data, nticks=200):
    string = 'tck_'
    waveforms = []
    #Here I extract a column in each iteration and append to list
    for i in range(nticks):
        waveforms.append(data[string+str(i)].astype(np.int16))
    #convert to numpy ndarray
    waveforms = np.array(waveforms).astype(np.int16)
    #since raws and columns are inverted we need to transpose it
    return np.transpose(waveforms)

# takes full raw data and returns waveform of length nticks
# only keeps waves at a desired adc count 
def get_std_waveforms(data_noisy, data_clean, nticks=200, min_adc=5):
    #Extract and scale waveform data (passthrough rn)
    raw_waveforms_noisy = extract_wave(data_noisy, nticks)
    raw_waveforms_clean = extract_wave(data_clean, nticks)
    #print('before adc filter: ', raw_waveforms_noisy.shape, raw_waveforms_clean.shape)

    noisy_ = []
    clean_ = []

    for i, wave in enumerate(raw_waveforms_clean):
        if max(wave) >= min_adc:
            noisy_.append(raw_waveforms_noisy[i])
            clean_.append(wave)
    
    del raw_waveforms_noisy, raw_waveforms_clean

    noisy_ = np.array(noisy_)
    clean_ = np.array(clean_)

    #print('after adc filter: ', noisy_.shape, clean_.shape)
    #print(raw_waveforms) 
    #scaled_waveforms = waveform_scaler.fit_transform(raw_waveforms)
    return noisy_, clean_



In [4]:
import tarfile
import os

# nuCC
dir_noisy_nu_cc = '/home/vlian/Workspace/train_dune_lartpc_v2/new_aug_10/nu_cc/'+wireplane+'/noisy_signal/'  # Directory to extract files
dir_clean_nu_cc = '/home/vlian/Workspace/train_dune_lartpc_v2/new_aug_10/nu_cc/'+wireplane+'/clean_signal/'  # Directory to extract files

noisy_names_cc = os.listdir(dir_noisy_nu_cc)
noisy_names_cc = sorted(noisy_names_cc)


clean_names_cc = os.listdir(dir_clean_nu_cc)
clean_names_cc = sorted(clean_names_cc)

# nuES
dir_noisy_nu_es = '/home/vlian/Workspace/train_dune_lartpc_v2/new_aug_10/nu_es/'+wireplane+'/noisy_signal/'  # Directory to extract files
dir_clean_nu_es = '/home/vlian/Workspace/train_dune_lartpc_v2/new_aug_10/nu_es/'+wireplane+'/clean_signal/'  # Directory to extract files

noisy_names_es = os.listdir(dir_noisy_nu_es)
noisy_names_es = sorted(noisy_names_es)

clean_names_es = os.listdir(dir_clean_nu_es)
clean_names_es = sorted(clean_names_es)


In [5]:
print(len(noisy_names_cc), len(clean_names_cc))
print(len(noisy_names_es), len(clean_names_es))

52 52
188 188


In [6]:
print(noisy_names_cc[0], '---', clean_names_cc[0])
print(noisy_names_es[0], '---', clean_names_es[0])

snb-nucc-en0-U-signal-3363366-0-0.npy --- snb-nucc-en0-U-clnsig-3363366-0-0.npy
snb-nues-en0-U-signal-63647624-0-0.npy --- snb-nues-en0-U-clnsig-63647624-0-0.npy


### seperate by energy

In [7]:
def get_wave_by_ENRG(energy_idx, noisy_filenames, clean_filenames, interaction_type=True):
    if interaction_type:
        noisy_path = dir_noisy_nu_cc
        clean_path = dir_clean_nu_cc
    else:
        noisy_path = dir_noisy_nu_es
        clean_path = dir_clean_nu_es

    file_names_noisy = [file for file in noisy_filenames if 'en'+str(energy_idx) in file ]
    file_names_clean = [file for file in clean_filenames if 'en'+str(energy_idx) in file ]

    noisy_waveforms = np.empty((0, 200))
    clean_waveforms = np.empty((0, 200))

    for i, file_name in enumerate(file_names_noisy):
        noisy_file_path = os.path.join(noisy_path, file_name)
        clean_file_path = os.path.join(clean_path, file_names_clean[i])
        

        noisy = np.load(noisy_file_path)
        clean = np.load(clean_file_path)

        noisy_wf, clean_wf = get_std_waveforms(noisy, clean, nticks=200, min_adc=5)
        
        noisy_waveforms = np.concatenate((noisy_waveforms, noisy_wf))
        clean_waveforms = np.concatenate((clean_waveforms, clean_wf))

    return [noisy_waveforms, clean_waveforms]


In [8]:
all_waveforms_nu_CC = []
all_waveforms_nu_ES = []
for i in range(10):
    all_waveforms_nu_CC.append(get_wave_by_ENRG(i, noisy_names_cc, clean_names_cc))
    all_waveforms_nu_ES.append(get_wave_by_ENRG(i, noisy_names_es, clean_names_es, False))

In [9]:
for i, en in enumerate(all_waveforms_nu_CC):
    print('en',i,':', 'nu_CC',en[0].shape, en[1].shape,'---','nu_ES', all_waveforms_nu_ES[i][0].shape, all_waveforms_nu_ES[i][1].shape)

en 0 : nu_CC (4452, 200) (4452, 200) --- nu_ES (7035, 200) (7035, 200)
en 1 : nu_CC (4497, 200) (4497, 200) --- nu_ES (6468, 200) (6468, 200)
en 2 : nu_CC (4700, 200) (4700, 200) --- nu_ES (6402, 200) (6402, 200)
en 3 : nu_CC (4794, 200) (4794, 200) --- nu_ES (6251, 200) (6251, 200)
en 4 : nu_CC (4782, 200) (4782, 200) --- nu_ES (6271, 200) (6271, 200)
en 5 : nu_CC (5995, 200) (5995, 200) --- nu_ES (6398, 200) (6398, 200)
en 6 : nu_CC (6121, 200) (6121, 200) --- nu_ES (6774, 200) (6774, 200)
en 7 : nu_CC (5995, 200) (5995, 200) --- nu_ES (6092, 200) (6092, 200)
en 8 : nu_CC (6043, 200) (6043, 200) --- nu_ES (6919, 200) (6919, 200)
en 9 : nu_CC (6883, 200) (6883, 200) --- nu_ES (6584, 200) (6584, 200)


In [10]:
#en = 9
#for i in range(20):
#    fig = plt.figure(figsize=(8,2))
#    plt.plot(all_waveforms_nu_ES[en][0][i])
#    plt.plot(all_waveforms_nu_ES[en][1][i])
#    plt.show()

### Load Noise

In [11]:
noise_path = '/home/vlian/Workspace/train_dune_lartpc_v2/noise/'+wireplane+'/'
noise_filenames = sorted([f for f in listdir(noise_path) if (isfile(join(noise_path, f)) and wireplane in f)])
combined_noise = np.concatenate([np.load(noise_path+fname, mmap_mode='r') for fname in noise_filenames])

noise_waveforms = extract_wave(combined_noise)
roi_truth_noise = np.zeros(noise_waveforms.shape[0]) # for roi-finding
print(noise_waveforms.shape, roi_truth_noise.shape)

(100000, 200) (100000,)


### TEST

In [12]:
model_5_10_mean = np.load('../models_scales/mean_5_10' + wireplane + '_nu.npy')
model_5_10_std = np.load('../models_scales/scale_5_10' + wireplane + '_nu.npy')

model_5_15_mean = np.load('../models_scales/mean_5_15' + wireplane + '_nu.npy')
model_5_15_std = np.load('../models_scales/scale_5_15' + wireplane + '_nu.npy')

model_5_18_mean = np.load('../models_scales/mean_5_18' + wireplane + '_nu.npy')
model_5_18_std = np.load('../models_scales/scale_5_18' + wireplane + '_nu.npy')

model_60k_mean = np.load('../models_scales/mean_60k' + wireplane + '_nu.npy')
model_60k_std = np.load('../models_scales/scale_60k' + wireplane + '_nu.npy')

In [13]:
scalers = [[model_5_10_mean, model_5_10_std], [model_5_15_mean, model_5_15_std], 
           [model_5_18_mean, model_5_18_std], [model_60k_mean, model_60k_std]]

In [14]:
model_5_10 = load_model('../ROI_ar39_models/model_5_10' + wireplane + 'plane_nu_ROI.h5')

model_5_15 = load_model('../ROI_ar39_models/model_5_15' + wireplane + 'plane_nu_ROI.h5')

model_5_18 = load_model('../ROI_ar39_models/model_5_18' + wireplane + 'plane_nu_ROI.h5')

model_60k = load_model('../ROI_ar39_models/model_60k' + wireplane + 'plane_nu_ROI.h5')

2023-08-11 04:39:49.486455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-11 04:39:49.519636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-11 04:39:49.520019: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [15]:
models = [model_5_10, model_5_15, model_5_18, model_60k]

In [16]:
def eval_model(idx, cnn_min):
    model_idx = idx

    noise_scaled = (noise_waveforms-scalers[model_idx][0])/scalers[model_idx][1]
    infer = models[model_idx].predict(noise_scaled, verbose=0)
    
    return (len([i for i in infer if i > cnn_min])/len(infer))*100

In [17]:
for i in range(4):
    print(round(100-eval_model(i, 0.999), 5), round(100-eval_model(i, 0.94), 5))

2023-08-11 04:39:52.071369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


100.0 99.514
99.995 99.456
99.997 99.609
99.997 99.735


In [18]:
def eval_model_en(en_group_waveforms, model_idx, cnn_min):

    waveforms_scaled = (en_group_waveforms-scalers[model_idx][0])/scalers[model_idx][1]
    infer = models[model_idx].predict(waveforms_scaled, verbose=0)
    
    return (len([i for i in infer if i > cnn_min])/len(infer))*100

In [19]:
len(all_waveforms_nu_ES)

10

In [20]:
for i, signals_at_en in enumerate(all_waveforms_nu_CC):
    print('en:', i)
    for j in range(4):
        #print('    model:', j)
        print(round(eval_model_en(signals_at_en[0], j, 0.999), 3), round(eval_model_en(signals_at_en[0], j, 0.94), 3))
    print('--------------')

en: 0
49.888 67.341
53.706 68.082
52.83 66.487
52.291 64.937
--------------
en: 1
58.528 72.359
61.597 72.893
61.152 71.492
60.618 70.803
--------------
en: 2
62.553 75.085
65.404 75.426
64.681 74.468
64.468 73.447
--------------
en: 3
65.54 77.806
68.002 77.931
67.73 77.013
67.376 76.095
--------------
en: 4
67.127 78.545
69.573 78.775
69.092 77.792
68.465 76.955
--------------
en: 5
70.158 81.168
72.744 81.435
72.427 80.567
71.943 79.9
--------------
en: 6
70.838 80.036
72.864 80.199
72.537 79.677
72.292 78.778
--------------
en: 7
72.11 81.918
74.012 82.385
73.745 81.568
73.294 80.717
--------------
en: 8
73.854 82.294
75.492 82.277
75.261 81.549
75.046 80.87
--------------
en: 9
80.604 86.924
81.839 87.099
81.65 86.619
81.505 86.038
--------------


In [21]:
#HERE

### Denoising Autoencoder

In [22]:
model_AE_check = load_model('../../../archive/AutoEncoder-Current/models/model_AE_2048_no_pooling'+wireplane+'plane_nu.h5')
mean_check = np.load('/home/vlian/Workspace/LArTPC-1DCNN-AutoEncoder/Neutrino-Trained-New-Dataset/archive/AutoEncoder-Current/models/saved_models/AE_mean_'+wireplane+'.npy')
std_check = np.load('/home/vlian/Workspace/LArTPC-1DCNN-AutoEncoder/Neutrino-Trained-New-Dataset/archive/AutoEncoder-Current/models/saved_models/AE_std_'+wireplane+'.npy')

In [23]:
print(mean_check, std_check)

-0.0857434375 10.171227500917754


In [24]:
nu_CC_energy = {0: '.028-5.50 MeV',
                1: '5.50-7.60 MeV',
                2: '7.60-10.0 MeV',
                3: '10.0-12.0 MeV',
                4: '12.0-15.0 MeV',
                5: '15.0-17.0 MeV',
                6: '17.0-20.0 MeV',
                7: '20.0-24.0 MeV',
                8: '24.0-29.0 MeV',
                9: '29.0-85.0 MeV'
                }

nu_ES_energy = {0: '0.005-0.010 GeV',
                1: '0.010-0.013 GeV',
                2: '0.013-0.016 GeV',
                3: '0.016-0.019 GeV',
                4: '0.019-0.021 GeV',
                5: '0.021-0.024 GeV',
                6: '0.024-0.027 GeV',
                7: '0.027-0.031 GeV',
                8: '0.031-0.036 GeV',
                9: '0.036-0.079 GeV',

}


In [25]:
def make_single_pdf(x, y, predicted, interaction,energy, energy_range, wave_idx, pg_num):

    fig, axs = plt.subplots(3,2, figsize=(20, 12), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .375, wspace=.1)

    axes = axs.ravel()

    for i in range(6):
        index_ = i + wave_idx
        wave_idx = index_
        axes[i].set_title(interaction + ': ' + energy_range + ' --- (peak adc: ' + str(max(y[wave_idx])) + ')')
        axes[i].plot(x[wave_idx], color='black', alpha=0.3, label='input')
        axes[i].plot(y[wave_idx], color='blue', label='target')
        axes[i].plot(predicted[wave_idx], color='m', label='prediction')
        axes[i].legend(fontsize=12)
    


    plt.savefig('./plots/tmp_u/tmp' +str(pg_num) + '.pdf',
                dpi=300,
                bbox_inches='tight', pad_inches=0.75)
    plt.close()

    return wave_idx

# creates and merges pdf, removes all single page pdfs from tmp folder
def make_complete_pdf(x, y, predicted, interaction, energy, energy_range, num_pages):
    wave_idx_ = 0
    page_num = 0

    while page_num < num_pages:
        wave_idx_ = make_single_pdf(x, y, predicted, interaction, energy, energy_range, wave_idx_, page_num) + 1
        page_num += 1

    merger = PdfMerger()
    path = './plots/tmp_u/'
    pdf_files = [path+f for f in listdir(path) if (isfile(join(path, f)))]
    #print(pdf_files)
    for pdf_file in pdf_files:
        #Append PDF files
        merger.append(pdf_file)
    #merger.write('pdfs/plts_tmp/plts_' + wireplane + '_cnn_'+str(int(min_cnn*100)) + '-' + str(int(max_cnn*100)) + '_' + str(num_pages) +  'pages.pdf')
    merger.write('./plots/'+wireplane+'/'+interaction+'/plt_U_plots_'+interaction+'_en'+str(energy)+'.pdf')
    merger.close()

    for file in pdf_files:
        os.remove(file)

In [26]:
def roi_ae(data_set, energy, energy_range,interaction, roi_model, ae_model, roi_scalers, ae_scalers):
    waveform_noisy = data_set[energy][0]
    print('energy en' + str(energy), energy_range)
    noisy_wave_scaled_ROI = (waveform_noisy-roi_scalers[0])/roi_scalers[1]
    noisy_wave_scaled_AE = (waveform_noisy-ae_scalers[0])/ae_scalers[1]
    
    waveform_clean = data_set[energy][1]
    clean_wave_scaled = (waveform_clean-ae_scalers[0])/ae_scalers[1]
    counter = 0

    noisy = np.empty((0, 200))
    clean = np.empty((0, 200))
    predicted = np.empty((0, 200))

    for i in trange(len(waveform_noisy)):
        wave_roi = noisy_wave_scaled_ROI[i:i+1]
        if roi_model.predict(wave_roi, verbose=0) > 0.999:
            wave_AE = noisy_wave_scaled_AE[i:i+1]
            if max(waveform_clean[i:i+1][0]) < 1000:
                ae_pred = ae_model.predict(wave_AE, verbose=0)
                ae_pred = ae_pred.reshape(ae_pred.shape[0], ae_pred.shape[1])
                pred = ae_pred*ae_scalers[1] + ae_scalers[0]
                
                noisy_wf = waveform_noisy[i]
                noisy_wf = noisy_wf.reshape(1, 200)
                noisy = np.concatenate((noisy, noisy_wf))
                #print('debug: ', waveform_clean[i:i+1].shape, clean.shape)
                clean = np.concatenate((clean, waveform_clean[i:i+1]))
                #print('debug: ', pred.shape, predicted.shape)
                predicted = np.concatenate((predicted, pred))
    for j in range(10):
        noisy, clean, predicted = shuffle(noisy, clean, predicted)
    
    
    make_complete_pdf(noisy, clean, predicted, interaction, energy, energy_range, 50)
    print('------------------------')
    perentage_ = (len(noisy)/len(waveform_noisy))*100
    print(len(noisy), ':', len(waveform_noisy), '%-> ', round(perentage_, 3))
    
    

In [27]:
len(all_waveforms_nu_CC[0][0])

4452

In [28]:
for energy_ in range(10):
    roi_ae(all_waveforms_nu_CC, energy_, nu_ES_energy[energy_], 'nuCC', model_60k, model_AE_check, [model_60k_mean, model_60k_std], [mean_check, std_check])

energy en0 0.005-0.010 GeV


100%|██████████| 4452/4452 [05:26<00:00, 13.63it/s]


------------------------
2328 : 4452 %->  52.291
energy en1 0.010-0.013 GeV


100%|██████████| 4497/4497 [05:40<00:00, 13.19it/s]


------------------------
2726 : 4497 %->  60.618
energy en2 0.013-0.016 GeV


...

------------------------
4394 : 5995 %->  73.294
energy en8 0.031-0.036 GeV


100%|██████████| 6043/6043 [08:19<00:00, 12.09it/s]


------------------------
4535 : 6043 %->  75.046
energy en9 0.036-0.079 GeV


100%|██████████| 6883/6883 [09:53<00:00, 11.60it/s]


------------------------
5610 : 6883 %->  81.505


In [29]:
##